In [0]:
# Before executing this bash cell you need to upload all the zip files 
# gracefully prepared by me into the /content directory.
# Then execute this cell.
# It'll create a data/train relative directory and unzip the uploaded files.

%%shell
rm -r /content/data/
mkdir -p /content/data/train

for i in {0..11}
do
  unzip "/content/train$i.zip" -d /content/data/train
  unzip "/content/train0$i.zip" -d /content/data/train
done

In [0]:
import pandas as pd
from glob import glob
import os
from PIL import Image
import numpy as np
import random

In [0]:
#%tensorflow_version 2.x

from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

random.seed(34)

Using TensorFlow backend.


In [0]:
# Скачайте полные данные отсюда https://www.kaggle.com/c/dogs-vs-cats/data (примерно 820мб)
# У вас должно быть две папки train и test. Однако в данном задании мы не будем использвать test выборку
# Функцию для чтения данных мы реализовали за вас

def read_dataset(path):
    X = []
    y = []
    
    image_paths_list = glob(os.path.join(path, 'train', '*.jpg'))
    image_paths_sample = random.sample(image_paths_list, 10000)

    for image_path in image_paths_sample:
        image_name = os.path.basename(image_path)
        image_name_parts = image_name.split('.')
        label = image_name_parts[0] if len(image_name_parts) == 3 else None

        if label:
            y.append(int(label == 'cat'))
            
        
        x = image.img_to_array(image.load_img(image_path, target_size=(224, 224)))
        x = preprocess_input(x)
            
        X.append(x)
            
    return np.array(X), np.array(y)

Перепишите код read_dataset() таким образом чтобы читать изображения пачками по 1000 и сразу применять предобученный ResNet на них, сохраняя только полученные вектора как множество X. Примените эту функцию на всём датасете 25000 изображений.

In [0]:
from tqdm import tqdm_notebook

train_directory = "data/"
batch_size = 1000
images_number = 10000

def read_dataset(path):
    """ X should be numpy array from the start. Otherwise it takes too much memory.
    """

    X = np.array([])
    y = np.array([])
    
    image_paths_list = glob(os.path.join(path, 'train', '*.jpg'))
    image_paths_list = random.sample(image_paths_list, images_number)
    
    np.random.shuffle(image_paths_list)
    
    # split into array of batch_size length arrays
    splitted_image_paths_list = np.split(np.array(image_paths_list), int(len(image_paths_list) / batch_size))
    
    for image_path_batch in tqdm_notebook(splitted_image_paths_list, desc='Batches'):
        x_batch = []
        y_batch = []
        
        for image_path in tqdm_notebook(image_path_batch, desc='Current batch', leave=False):
            image_name = os.path.basename(image_path)
            image_name_parts = image_name.split('.')
            label = image_name_parts[0] if len(image_name_parts) == 3 else None
            
            if label:
                y_batch.append(int(label == 'cat'))
            
            x = image.img_to_array(image.load_img(image_path, target_size=(224, 224)))#, dtype=np.uint8)

            '''if x_batch.size == 0:
              x_batch = x[None,:,:,:]
            else:
              x_batch = np.concatenate([x_batch, x[None,:,:,:]], axis=0)'''
            x_batch.append(x)
        

        x_batch = preprocess_input(np.array(x_batch))
        y_batch = np.array(y_batch)
        
        if X.size == 0:
          X = x_batch
          y = y_batch
        else:
          X = np.concatenate([X, x_batch], axis=0)
          y = np.concatenate([y, y_batch], axis=0)
    
    return X, y

In [0]:
# Используйте функцию read_dataset чтобы получить обучающую выборку
features, labels = read_dataset("/content/data")

# Проверьте размерности загруженных данных
print(features.shape)
print(labels.shape)

In [0]:
# с помощью функции train_test_split поделите выборку на train и test в отношении 70/30
# В качестве признаков используйте эмбеддинги, полученные на предыдущем шаге
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = 0.3)
print(features_train.shape)
print(labels_train.shape)
print(features_test.shape)
print(labels_test.shape)

(7000, 224, 224, 3)
(7000,)
(3000, 224, 224, 3)
(3000,)


**Что такое эмбеддинг в данном контексте? Как я понял, надо преобразовать картинку размерности (224, 244, 3) в одномерный массив длины m?  Не ясно при чем здесь ResNet**

In [0]:
# Создаём модель для получения эмбеддингов
model = ResNet50(weights='imagenet')

# Используя объект model постройте эмбеддинги для обучающей выборки
embeddings = model.predict(features_train)

Попытка классифицировать свою кошку с помощью ResNet.

In [0]:
!unzip /content/jerry.zip -d /content/data/my-pet

In [0]:
# check my own pet photos
def read_my_pet_dataset(path):
    X = []
    
    image_paths_list = glob(os.path.join(path, 'my-pet', '*.jpg'))

    for image_path in image_paths_list:
        image_name = os.path.basename(image_path)        
        x = image.img_to_array(image.load_img(image_path, target_size=(224, 224)))
        x = preprocess_input(x)
            
        X.append(x)
            
    return np.array(X)


pet_features = read_my_pet_dataset("data/")
print(pet_features.shape)

pet_predictions = model.predict(pet_features)
print(decode_predictions(pet_predictions, top=1))

(16, 224, 224, 3)
40960/35363 [==================================] - 0s 0us/step
[[('n02123394', 'Persian_cat', 0.58622897)], [('n02123394', 'Persian_cat', 0.3944722)], [('n02123394', 'Persian_cat', 0.28453523)], [('n02102973', 'Irish_water_spaniel', 0.12897159)], [('n01622779', 'great_grey_owl', 0.3220467)], [('n02441942', 'weasel', 0.25759768)], [('n02441942', 'weasel', 0.4639576)], [('n02497673', 'Madagascar_cat', 0.21311994)], [('n02123394', 'Persian_cat', 0.31379968)], [('n02127052', 'lynx', 0.41493312)], [('n02113624', 'toy_poodle', 0.7856534)], [('n04399382', 'teddy', 0.16075166)], [('n02127052', 'lynx', 0.24749047)], [('n02127052', 'lynx', 0.27399546)], [('n01877812', 'wallaby', 0.16427185)], [('n02093647', 'Bedlington_terrier', 0.05776652)]]


Создайте объект XGBClassifier со стандартными параметрами

https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBClassifier

Обучите его

Неправильная размерность в fit. Скорее всего надо преобразовать картинки

In [0]:
# Получите предсказания на тестовом и на обучающем множестве
clf = XGBClassifier()

clf.fit(features_train, labels_train)
predicted_test = clf.predict(features_test)

print(accuracy_score(labels_test, predicted_test))

ValueError: ignored

In [0]:
# С помощью функции accuracy_score оцените результаты


Почитайте про параметры XGBoost здесь: https://sites.google.com/view/lauraepp/parameters
Сделайте тюнинг гиперпараметров для XGBoost:
1. Подберите максимальный num_iterations при котором скор на валидации близок к максимальному, а скорость обучения ещё приемлимая
2. Для заданного num_iterations подберите подходящий learning rate
3. С помощью библиотеки hyperopt подберите остальные гиперпараметры при фиксимрованных num_iterations и learning_rate. Не подбирайте параметр early_stopping

Оцените важность признаков с помощью поля feature_importances_

Пользуясь списком https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a посмотрите, какие классы оказались наиболее важными (со значениями importance больше 0)
    
Изменяйте параметры, чтобы оставить только действительно важные признаки